In [1]:
import pandas as pd
from clickhouse_driver import Client
from datetime import datetime

In [2]:
# Подключение к серверу ClickHouse
client = Client('localhost')

In [18]:
# 1. Выполнение базового запроса для получения всех записей из таблицы 'crashes'
records = client.execute('SELECT * FROM town_cary.crashes')
df_records = pd.DataFrame(records, columns=['id', 'location_description', 'rdfeature', 'rdsurface', 'rdcondition', 'lightcond', 'weather', 'crash_date', 'year', 'fatalities', 'injuries', 'month'])
df_records.head()

,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,48436,288 FEET FROM P.V.A. (3710 DAVIS DR),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627164009,2021,No,No,7
1,48438,495 FEET FROM NC 54 HWY (CHAPEL HILL RD),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627140540,2021,No,No,7
2,48442,90 FEET FROM PVA OF 1405 KILDAIRE FARM RD,"DRIVEWAY, PUBLIC",SMOOTH ASPHALT,DRY,DAYLIGHT,CLOUDY,1627303062,2021,No,No,7
3,48464,40 FEET FROM P.V.A. (200 WESTOVER HILLS DR),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627295951,2021,No,No,7
4,48485,CARY,RELATED TO INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627472999,2021,No,No,7


In [19]:
# 2. Фильтрация записей на основе критериев
# 2.1. Равенство: Получение записей с fatalities, равным 'Yes'
fatal_records = client.execute("SELECT * FROM town_cary.crashes WHERE fatalities = 'Yes'")
df_fatal_records = pd.DataFrame(fatal_records, columns=df_records.columns)
df_fatal_records.head()

,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,28642,0 FEET FROM SR 4067 (REGENCY PKWY),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1491504285,2017,Yes,No,4
1,28953,.28 MILES FROM US 1 HWY,NO SPECIAL FEATURE,GROOVED CONCRETE,DRY,DARK - ROADWAY NOT LIGHTED,CLEAR,1493656200,2017,Yes,No,5
2,29042,.28 MILES FROM US 1 HWY,,,,,,1493536320,2017,Yes,No,4
3,29517,.28 MILES FROM US 1 HWY,,,,,,1493611380,2017,Yes,No,5
4,30506,0 FEET FROM HWY 54,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DARK - LIGHTED ROADWAY,CLEAR,1504584300,2017,Yes,No,9


In [6]:
# 2.2. Равенство: Получение записей с годом, равным '2021'
records_2021 = client.execute("SELECT * FROM town_cary.crashes WHERE year == '2021'")
df_2021_records = pd.DataFrame(non_2021_records, columns=df_records.columns)
df_2021_records.head()

Non-2021 records:


,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,48436,288 FEET FROM P.V.A. (3710 DAVIS DR),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627164009,2021,No,No,7
1,48438,495 FEET FROM NC 54 HWY (CHAPEL HILL RD),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627140540,2021,No,No,7
2,48442,90 FEET FROM PVA OF 1405 KILDAIRE FARM RD,"DRIVEWAY, PUBLIC",SMOOTH ASPHALT,DRY,DAYLIGHT,CLOUDY,1627303062,2021,No,No,7
3,48464,40 FEET FROM P.V.A. (200 WESTOVER HILLS DR),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627295951,2021,No,No,7
4,48485,CARY,RELATED TO INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627472999,2021,No,No,7
...,...,...,...,...,...,...,...,...,...,...,...,...
4172,51356,.12 MILES FROM SR 3112 (CARY PKWY),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DARK - LIGHTED ROADWAY,CLEAR,1639795675,2021,No,No,12
4173,51488,320 FEET FROM SR 1652 (N HARRISON AV),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627706940,2021,No,No,7
4174,51489,265 FEET FROM WESTON PKWY,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DARK - LIGHTED ROADWAY,CLEAR,1638164820,2021,No,No,11
4175,51490,0 FEET FROM SR 1613 (DAVIS DR),FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1620619200,2021,No,No,5


In [14]:
# 2.3. Диапазон: Выборка записей с датой аварии между двумя временными метками
start_timestamp = 1627560000
end_timestamp = 1627565000
range_records = client.execute(f"SELECT * FROM town_cary.crashes WHERE crash_date BETWEEN {start_timestamp} AND {end_timestamp}")
df_range_records = pd.DataFrame(range_records, columns=df_records.columns)
df_range_records

Records in range:


,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
1,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
2,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
3,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
4,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
5,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
6,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1627564933,2021,No,No,7
7,48499,420 FEET FROM PVA OF 3480 SR 1300 (KILDAIRE,"DRIVEWAY, PUBLIC",COARSE ASPHALT,DRY,DAYLIGHT,CLEAR,1627562821,2021,No,No,7


In [16]:
# Изменение формата даты
df_range_records['crash_date'] = df_range_records['crash_date'].apply(lambda ts: datetime.fromtimestamp(ts).strftime('%Y-%m-%d'))

df_range_records

,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
1,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
2,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
3,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
4,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
5,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
6,48493,15 FEET FROM US 64 HWY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7
7,48499,420 FEET FROM PVA OF 3480 SR 1300 (KILDAIRE,"DRIVEWAY, PUBLIC",COARSE ASPHALT,DRY,DAYLIGHT,CLEAR,2021-07-29,2021,No,No,7


In [8]:
# 3. Сортировка записей на основе одного или нескольких полей
# 3.1. Сортировка записей по дате аварии в порядке убывания
sorted_records = client.execute("SELECT * FROM town_cary.crashes ORDER BY crash_date DESC")
df_sorted_records = pd.DataFrame(sorted_records, columns=df_records.columns)
df_sorted_records.head()

Sorted records:


,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,51971,CARY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1653057660,2022,No,No,5
1,51964,.13 MILES FROM PVA 4980 CARY GLEN BLVD,NO SPECIAL FEATURE,COARSE ASPHALT,DRY,DAYLIGHT,CLEAR,1653014760,2022,No,No,5
2,51965,CARY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1653001763,2022,No,Yes,5
3,51970,95 FEET FROM S ACADEMY ST,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1652985894,2022,No,Yes,5
4,51967,91 FEET FROM NC 55 HWY,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1652981136,2022,No,No,5
...,...,...,...,...,...,...,...,...,...,...,...,...
23654,27461,CARY,RELATED TO INTERSECTION,SMOOTH ASPHALT,WET,DARK - LIGHTED ROADWAY,RAIN,1483319008,2017,No,No,1
23655,27460,CARY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,WET,DARK - LIGHTED ROADWAY,RAIN,1483313496,2017,No,No,1
23656,27466,CARY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,WET,DARK - LIGHTED ROADWAY,RAIN,1483307316,2017,No,No,1
23657,27459,380 FEET FROM NC 54 HWY,NO SPECIAL FEATURE,SMOOTH ASPHALT,WET,DAYLIGHT,RAIN,1483291860,2017,No,No,1


In [9]:
# 3.2. Сортировка записей по году в порядке возрастания и по дате аварии в порядке убывания
multi_sorted_records = client.execute("SELECT * FROM town_cary.crashes ORDER BY year ASC, crash_date DESC")
df_multi_sorted_records = pd.DataFrame(multi_sorted_records, columns=df_records.columns)
df_multi_sorted_records.head()

Multi-sorted records:


,id,location_description,rdfeature,rdsurface,rdcondition,lightcond,weather,crash_date,year,fatalities,injuries,month
0,32250,60 FEET FROM PVA 440 CROSSROADS BLVD,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1514768400,2017,No,No,12
1,32193,200 FEET FROM VASHON CT,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLOUDY,1514758090,2017,No,No,12
2,32192,318 FEET FROM P.V.A. (2080 KILDAIRE FARM RD),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLOUDY,1514753358,2017,No,No,12
3,32191,CARY,FOUR-WAY INTERSECTION,SMOOTH ASPHALT,DRY,DAYLIGHT,CLOUDY,1514740620,2017,No,No,12
4,32194,.1 MILES FROM P.V.A. 2021 WALNUT ST (TARGET),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1514727040,2017,No,No,12
...,...,...,...,...,...,...,...,...,...,...,...,...
23654,50540,CARY,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1641061589,2022,No,No,1
23655,50468,200 FEET FROM P.V.A. (125 WESTON PKWY),NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLEAR,1641051000,2022,No,No,1
23656,50448,300 FEET FROM ACADEMY ST,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DAYLIGHT,CLOUDY,1641048099,2022,No,No,1
23657,50461,.098 MILES FROM WESTON PKWY,NO SPECIAL FEATURE,SMOOTH ASPHALT,DRY,DARK - LIGHTED ROADWAY,CLOUDY,1641032884,2022,No,Yes,1


In [10]:
# 4. Агрегировать записи с помощью таких функций, как count, sum и avg.
# 4.1. Подсчет общего количества записей
count_records = client.execute("SELECT COUNT(*) FROM town_cary.crashes")
print("Общее количество записей:", count_records[0][0])

Total records count: 23659


In [11]:
# 4.2. Подсчет количества записей за каждый год
year_count_records = client.execute("SELECT year, COUNT(*) FROM town_cary.crashes GROUP BY year")
df_year_count_records = pd.DataFrame(year_count_records, columns=['year', 'count'])
df_year_count_records

Year-wise records count:


,year,count
0,2019,5035
1,2021,4177
2,2017,4731
3,2022,1492
4,2018,5024
5,2020,3200


In [12]:
# 4.3. Рассчитайте среднюю дату ДТП для записей с injuries = 'Yes'
avg_crash_date = client.execute("SELECT AVG(crash_date) FROM town_cary.crashes WHERE injuries = 'Yes'")
print("Средняя дата ДТП для записей с травмами:", avg_crash_date[0][0])

Average crash_date for records with injuries: 1564574122.2956831


In [20]:
date_object = datetime.fromtimestamp(avg_crash_date[0][0])
date_string = date_object.strftime('%Y-%m-%d')

print("Timestamp:", timestamp)
print("Дата:", date_string)

Timestamp: 1564574122
Date: 2019-07-31
